In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
import os
from tqdm import tqdm

from holodeck import plot, detstats, utils
from holodeck.constants import YR, MSOL, MPC, GYR
import holodeck as holo

Set up

In [ ]:
SHAPE = None
NREALS = 500
# NREALS = 20
NFREQS = 40
NLOUDEST = 10

BUILD_ARRAYS = False
SAVEFIG = True
TOL=0.01
MAXBADS=5

NVARS = 21
# NVARS = 6

NPSRS = 40
NSKIES = 100
# NSKIES = 15

MEDIAN=False
AVG=True

Get param names

In [ ]:
pspace = holo.param_spaces.PS_Uniform_09B(holo.log, nsamples=1, sam_shape=SHAPE, seed=None)
param_names = pspace.param_names

Get data and dets file functions

In [ ]:
def get_data(
        target, nvars=NVARS, nreals=NREALS, nskies=NSKIES, shape=SHAPE, red_gamma = None, red2white=None,
    path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz'     
):

    load_data_from_file = path+f'/{target}_v{nvars}_r{nreals}_shape{str(shape)}/data_params.npz' 
    load_dets_from_file = path+f'/{target}_v{nvars}_r{nreals}_shape{str(shape)}/detstats_s{nskies}' 
    
    if red_gamma is not None and red2white is not None:
        load_dets_from_file = load_dets_from_file+f'_r2w{red2white:.1f}_rg{red_gamma:.1f}'
    else:
        load_dets_from_file = load_dets_from_file+f'_white'
    load_dets_from_file = load_dets_from_file+'.npz'

    if os.path.exists(load_data_from_file) is False:
        err = f"load data file '{load_data_from_file}' does not exist, you need to construct it."
        raise Exception(err)
    if os.path.exists(load_dets_from_file) is False:
        err = f"load dets file '{load_dets_from_file}' does not exist, you need to construct it."
        raise Exception(err)
    file = np.load(load_data_from_file, allow_pickle=True)
    data = file['data']
    params = file['params']
    file.close()

    file = np.load(load_dets_from_file, allow_pickle=True)
    dsdat = file['dsdat']
    file.close()

    return data, params, dsdat

In [ ]:
def _twiny_hz(ax, nano=True, fs=10, label=True, **kw):
    tw = ax.twinx()
    tw.grid(False)
    ylim = np.array(ax.get_ylim()) / YR
    if nano:
        ylim *= 1e9

    tw.set(ylim=ylim, yscale=ax.get_yscale())
    if label:
        label = plot.LABEL_GW_FREQUENCY_NHZ if nano else plot.LABEL_GW_FREQUENCY_YR
        tw.set_ylabel(label, fontsize=fs, **kw)
    return tw

### Build arrays

In [ ]:
if BUILD_ARRAYS:
    targets = [
        'gsmf_phi0',  'mmb_mamp_log10', 'hard_time',  
        'gsmf_mchar0_log10', 'mmb_scatter_dex', 'hard_gamma_inner']
    path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata'  
    for target in tqdm(targets):
        print(target)
        xx = [] # param
        yy = [] # frequency means

        # white noise only
        data, params, dsdat = get_data(target)
        freqs = data[0]['fobs_cents']
        freqs = np.repeat(freqs, NREALS*NSKIES*NLOUDEST).reshape(NFREQS, NREALS, NSKIES, NLOUDEST)

        for pp, par in enumerate(params):
            xx.append(params[pp][target])
            dpssi = dsdat[pp]['gamma_ssi']
            if MEDIAN:
                freq_mean = np.average(freqs, weights=dpssi, axis=(0,-1))
            elif AVG:
                freq_mean = np.average(freqs, weights=dpssi)
            yy.append(freq_mean)
        if MEDIAN:
            saveloc = path+f'/freq_means_{target}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}.npz'
        elif AVG:
            saveloc = path+f'/freq_avg_{target}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}.npz'
        np.savez(saveloc, xx_params = xx, yy_fmeans = yy)
# else:
#     path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata'  
#     file = np.load(path+f'/freq_means_{target}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}.npz')
#     xx = file['xx_params']
#     yy = file['yy_fmeans']
#     file.close()


In [ ]:
colors = [
    '#336948', '#9e5c41', '#2d839f',
    '#336948', '#9e5c41', '#2d839f',
]
path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata'  
   

targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
    'mmb_mamp_log10', 'mmb_scatter_dex',
    'hard_time', 'hard_gamma_inner'
    ]

ylabel = 'Frequency [yr$^{-1}$]'
ylabel_nHz = 'Frequency [nHz]'


fig, axs = plot.figax_double(nrows=2, ncols=3, sharey=True, sharex=False, xscale='linear')

fig.text(0.06, 0.5, ylabel, ha='right', va='center', rotation='vertical')
fig.text(1.02, 0.5, ylabel_nHz, ha='right', va='center', rotation='vertical')
plt.subplots_adjust(wspace=0)
for ii, ax in enumerate(axs.flatten()):
    ax.set_xlabel(plot.PARAM_KEYS[targets[ii]])
    # if ii == 0 or ii == 3:
    #     ax.set_ylabel(ylabel)
    file = np.load(path+f'/freq_avg_{targets[ii]}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}.npz')
    xx = file['xx_params']
    yy = file['yy_fmeans']*YR
    # print(xx.shape, yy.shape)
    # print(f"{yy.shape=}")
    ax.plot(xx, yy, color=colors[ii])
    if ii==2 or ii==5:
        _twiny_hz(ax, label=False, nano=True)
        

saveloc = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots'
savename = saveloc+'/favg_double.png'
fig.savefig(savename, dpi=100, bbox_inches='tight')

In [ ]:

targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
    'mmb_mamp_log10', 'mmb_scatter_dex',
    'hard_time', 'hard_gamma_inner']
colors = [
    '#336948', '#336948', 
    '#9e5c41', '#9e5c41', 
    '#2d839f', '#2d839f',
]
path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata'  
   

ylabel = 'Frequency [yr$^{-1}$]'
ylabel_nHz = 'Frequency [nHz]'
fig, axs = plot.figax_single(nrows=3, ncols=2, 
                             sharey=True, sharex=False, xscale='linear',
                             height=7)


for ii, ax in enumerate(axs.flatten()):
    ax.set_xlabel(plot.PARAM_KEYS[targets[ii]])
    # if ii == 0 or ii == 3:
    #     ax.set_ylabel(ylabel)
    file = np.load(path+f'/freq_avg_{targets[ii]}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}.npz')
    xx = file['xx_params']
    yy = file['yy_fmeans']*YR
    print(f"{yy.shape=}")
    ax.plot(xx, yy, color=colors[ii])
    if ii%2 == 1:
        _twiny_hz(ax, label=False)
# fig.tight_layout()
    # ax.tick_params(axis='x', labelrotation=45)

fig.text(0.02, 0.5, ylabel, ha='right', va='center', rotation='vertical')
fig.text(1.12, 0.5, ylabel_nHz, ha='right', va='center', rotation='vertical')
plt.subplots_adjust(wspace=0.05, hspace=0.35)
saveloc = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots'
savename = saveloc+'/favg_single.png'
fig.savefig(savename, dpi=100, bbox_inches='tight')

# Add GW only

In [ ]:
if BUILD_ARRAYS:
    targets = [
        'gsmf_phi0',  'mmb_mamp_log10', 
        'gsmf_mchar0_log10', 'mmb_scatter_dex', ]
    path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata'  
    for target in tqdm(targets):
        print(target)
        xx = [] # param
        yy = [] # frequency means

        # white noise only
        data, params, dsdat = get_data(target, path = '/Users/emigardiner/GWs/holodeck/output/anatomy_7GW')
        freqs = data[var]['fobs_cents']
        freqs = np.repeat(freqs, NREALS*NSKIES*NLOUDEST).reshape(NFREQS, NREALS, NSKIES, NLOUDEST)

        for pp, par in enumerate(params):
            xx.append(params[pp][target])
            dpssi = dsdat[pp]['gamma_ssi']
            if MEDIAN:
                freq_mean = np.average(freqs, weights=dpssi, axis=(0,-1))
            elif AVG:
                freq_mean = np.average(freqs, weights=dpssi)
            yy.append(freq_mean)
        if MEDIAN:
            saveloc = path+f'/freq_means_gw_{target}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}.npz'
        elif AVG:
            saveloc = path+f'/freq_avg_gw_{target}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}.npz'

        np.savez(saveloc,  xx_params = xx, yy_fmeans = yy)
# else:
#     path = '/Users/emigardiner/GWs/holodeck/output/anatomy_7GW/figdata'  
#     file = np.load(path+f'/freq_means_gw_{target}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}.npz')
#     xx_gw = file['xx_params']
#     yy_gw = file['yy_fmeans']
#     file.close()


In [ ]:

targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
    'mmb_mamp_log10', 'mmb_scatter_dex',
    'hard_time', 'hard_gamma_inner']
colors = [
    '#336948', '#336948', 
    '#9e5c41', '#9e5c41', 
    '#2d839f', '#2d839f',
]
path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata'  
   

ylabel = 'Frequency [yr$^{-1}$]'
ylabel_nHz = 'Frequency [nHz]'
fig, axs = plot.figax_single(nrows=3, ncols=2, 
                             sharey=True, sharex=False, xscale='linear',
                             height=7)


for ii, ax in enumerate(axs.flatten()):
    ax.set_xlabel(plot.PARAM_KEYS[targets[ii]])
    # if ii == 0 or ii == 3:
    #     ax.set_ylabel(ylabel)
    file = np.load(path+f'/freq_avg_{targets[ii]}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}.npz')
    xx = file['xx_params']
    yy = file['yy_fmeans']*YR
    print(f"{yy.shape=}")

    ax.plot(xx, yy, color=colors[ii], alpha=0.9)
    # for pp in [50,95]:
    #     percs = [50-pp/2, 50+pp/2]
    #     ax.fill_between(xx, *np.percentile(yy, percs, axis=-1), color=colors[ii], alpha=0.25)
    # plot.draw_med_conf_color(ax, xx, yy.reshape(NVARS, NREALS*NSKIES), color=colors[ii])

    print(f"{yy.shape=}")

    if ii<4:
        file = np.load(path+f'/freq_avg_gw_{targets[ii]}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}.npz')
        xx_gw = file['xx_params']
        yy_gw = file['yy_fmeans']*YR
    elif ii==4: # hard_time, get mean, hard_gamma_inner uses same
        yy_gw = np.repeat(yy_gw[10], NVARS)

    ax.plot(xx, yy_gw, color='k', alpha=0.75, linestyle='--')
    # for pp in [50,]:
    #     percs = [50-pp/2, 50+pp/2]
    #     ax.fill_between(xx, *np.percentile(yy_gw, percs, axis=-1), color='k', alpha=0.15, linestyle='--')
    
    if ii%2 == 1:
        _twiny_hz(ax, label=False)

    # print min and max freq
    print(f"{np.min(yy)=}, {np.max(yy)=}, {np.min(yy_gw)=}, {np.max(yy_gw)=}")
# fig.tight_layout()
    # ax.tick_params(axis='x', labelrotation=45)

fig.text(0.04, 0.5, ylabel, ha='right', va='center', rotation='vertical')
fig.text(1.09, 0.5, ylabel_nHz, ha='right', va='center', rotation='vertical')
plt.subplots_adjust(wspace=0.05, hspace=0.35)
saveloc = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots'
savename = saveloc+'/favg_gw_single.png'
fig.savefig(savename, dpi=100, bbox_inches='tight')

# Weighted STdev

$$ \bar{X} = \big( \sum_i W_i X_i  \big)/\sum_i W_i$$
$$ S^2 = \big(\sum_i W_i (X_i - \bar{X})^2\big) / \big( \frac{n-1}{n} \sum_i W_i  \big)

In [ ]:
def weighted_mean_variance(data, weights, debug=False, log=False):
    mean = np.sum(weights * data) / np.sum(weights)
    if debug: print(f"{mean=}")
    var2 = np.sum(weights * (data - mean)**2) 
    nn = data.size
    var2 /= (nn-1)/nn * np.sum(weights)
    if debug: print(f"{var2=}")
    return mean, var2

In [ ]:
if True:
    targets = [
        'gsmf_phi0',  'mmb_mamp_log10', 'hard_time',  
        'gsmf_mchar0_log10', 'mmb_scatter_dex', 'hard_gamma_inner']
    path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata'  
    for target in tqdm(targets):
        print(target)
        xx = [] # param
        yy = [] # frequency means
        yy_log = []
        vv = [] # variance
        vv_log = [] # variance of log frequency

        # white noise only
        data, params, dsdat = get_data(target)
        freqs = data[0]['fobs_cents']
        freqs = np.repeat(freqs, NREALS*NSKIES*NLOUDEST).reshape(NFREQS, NREALS, NSKIES, NLOUDEST)

        for pp, par in enumerate(params):
            xx.append(params[pp][target])
            dpssi = dsdat[pp]['gamma_ssi']
            mean, var2 = weighted_mean_variance(freqs, weights=dpssi)
            logmean, logvar2 = weighted_mean_variance(np.log10(freqs), weights=dpssi)
            
            var = np.sqrt(var2)
            logvar = np.sqrt(logvar2)

            yy.append(mean)
            yy_log.append(logmean)
            vv.append(var)
            vv_log.append(logvar)
        saveloc = path+f'/freq_meanvar_{target}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}.npz'
        np.savez(saveloc, xx = xx, yy = yy, vv = vv, yy_log = yy_log, vv_log=vv_log)


In [ ]:
if True:
    # GW only
    targets = [
        'gsmf_phi0',  'mmb_mamp_log10', 
        'gsmf_mchar0_log10', 'mmb_scatter_dex', ]
    path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata'  
    for target in tqdm(targets):
        print(target)
        xx = [] # param
        yy = [] # frequency means
        vv = [] # variance
        vv_log = [] # variance of log frequency
        yy_log = []

        # white noise only
        data, params, dsdat = get_data(target, path = '/Users/emigardiner/GWs/holodeck/output/anatomy_7GW')
        freqs = data[0]['fobs_cents']
        freqs = np.repeat(freqs, NREALS*NSKIES*NLOUDEST).reshape(NFREQS, NREALS, NSKIES, NLOUDEST)

        for pp, par in enumerate(params):
            xx.append(params[pp][target])
            dpssi = dsdat[pp]['gamma_ssi']
            mean, var2 = weighted_mean_variance(freqs, weights=dpssi)
            logmean, logvar2 = weighted_mean_variance(np.log10(freqs), weights=dpssi)
            var = np.sqrt(var2)
            logvar = np.sqrt(logvar2)
            yy.append(mean)
            vv.append(var)

            yy_log.append(logmean)
            vv_log.append(logvar)
        
        saveloc = path+f'/freq_meanvar_gw_{target}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}.npz'
        np.savez(saveloc,  xx = xx, yy = yy, vv = vv, yy_log = yy_log, vv_log = vv_log)

In [ ]:
xx = 5e-9
print(np.log10(10**xx *YR))
print(xx + np.log10(YR))

In [ ]:

targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
    'mmb_mamp_log10', 'mmb_scatter_dex',
    'hard_time', 'hard_gamma_inner']
colors = [
    '#336948', '#336948', 
    '#9e5c41', '#9e5c41', 
    '#2d839f', '#2d839f',
]
path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata'  
   

ylabel = 'Frequency [yr$^{-1}$]'
ylabel_nHz = 'Frequency [nHz]'
fig, axs = plot.figax_single(nrows=3, ncols=2, 
                             sharey=True, sharex=False, xscale='linear',
                             height=7)


for ii, ax in enumerate(axs.flatten()):
    ax.set_xlabel(plot.PARAM_KEYS[targets[ii]])
    # if ii == 0 or ii == 3:
    #     ax.set_ylabel(ylabel)
    file = np.load(path+f'/freq_meanvar_{targets[ii]}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}.npz')
    xx, yy, vv, yy_log, vv_log = file['xx'], file['yy'], file['vv'], file['yy_log'], file['vv_log']
    yy_min = 10**(yy_log - vv_log) *YR
    yy_max = 10**(yy_log + vv_log) *YR
    yy = 10**yy_log * YR
    # print(f"{yy.shape=}")

    ax.plot(xx, yy, color=colors[ii], alpha=0.9)
    # ax.errorbar(xx, yy, yerr=(yy_min, yy_max), color=colors[ii], alpha=0.9)
    ax.errorbar(xx, yy, yerr=(yy-yy_min, yy_max-yy), color=colors[ii], alpha=0.7, capsize=3)

    # for pp in [50,95]:
    #     percs = [50-pp/2, 50+pp/2]
    #     ax.fill_between(xx, *np.percentile(yy, percs, axis=-1), color=colors[ii], alpha=0.25)
    # plot.draw_med_conf_color(ax, xx, yy.reshape(NVARS, NREALS*NSKIES), color=colors[ii])

    # print(f"{yy.shape=}")

    if ii<4:
        file = np.load(path+f'/freq_meanvar_gw_{targets[ii]}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}.npz')
        xx_gw, yy_gw, vv_gw = file['xx'], file['yy']*YR, file['vv']*YR

    elif ii==4: # hard_time, get mean, hard_gamma_inner uses same
        yy_gw = np.repeat(yy_gw[10], NVARS)
        vv_gw = np.repeat(vv_gw[10], NVARS)

    ax.plot(xx, yy_gw, color='k', alpha=0.75, linestyle='--')
    # ax.errorbar()
    # for pp in [50,]:
    #     percs = [50-pp/2, 50+pp/2]
    #     ax.fill_between(xx, *np.percentile(yy_gw, percs, axis=-1), color='k', alpha=0.15, linestyle='--')
    
    if ii%2 == 1:
        _twiny_hz(ax, label=False)

    # print min and max freq
    print(f"{np.min(yy)=}, {np.max(yy)=}, {np.min(yy_gw)=}, {np.max(yy_gw)=}")
# fig.tight_layout()
    # ax.tick_params(axis='x', labelrotation=45)

fig.text(0.04, 0.5, ylabel, ha='right', va='center', rotation='vertical')
fig.text(1.09, 0.5, ylabel_nHz, ha='right', va='center', rotation='vertical')
plt.subplots_adjust(wspace=0.05, hspace=0.35)
# saveloc = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots'
# savename = saveloc+'/fmeanvar_gw_single.png'
# fig.savefig(savename, dpi=100, bbox_inches='tight')

In [ ]:

targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
    'mmb_mamp_log10', 'mmb_scatter_dex',
    'hard_time', 'hard_gamma_inner']
colors = [
    '#336948', '#336948', 
    '#9e5c41', '#9e5c41', 
    '#2d839f', '#2d839f',
]
path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata'  
   

ylabel = 'Frequency [yr$^{-1}$]'
ylabel_nHz = 'Frequency [nHz]'
fig, axs = plot.figax_single(nrows=3, ncols=2, 
                             sharey=True, sharex=False, xscale='linear',
                             height=7)


for ii, ax in enumerate(axs.flatten()):
    ax.set_xlabel(plot.PARAM_KEYS[targets[ii]])
    # if ii == 0 or ii == 3:
    #     ax.set_ylabel(ylabel)
    file = np.load(path+f'/freq_meanvar_{targets[ii]}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}.npz')
    xx, yy, vv, yy_log, vv_log = file['xx'], file['yy'], file['vv'], file['yy_log'], file['vv_log']
    yy_min = 10**(yy_log - vv_log) *YR
    yy_max = 10**(yy_log + vv_log) *YR
    yy = 10**yy_log * YR
    # print(f"{yy.shape=}")

    ax.plot(xx, yy, color=colors[ii], alpha=0.9)
    # ax.errorbar(xx, yy, yerr=(yy_min, yy_max), color=colors[ii], alpha=0.9)
    # ax.errorbar(xx, yy, yerr=(yy_min, yy_max), color=colors[ii], alpha=0.7, capsize=3)
    ax.scatter(xx, yy_min, color=colors[ii], alpha=0.7, marker=10)
    ax.scatter(xx, yy_max, color=colors[ii], alpha=0.7, marker=11)

    # for pp in [50,95]:
    #     percs = [50-pp/2, 50+pp/2]
    #     ax.fill_between(xx, *np.percentile(yy, percs, axis=-1), color=colors[ii], alpha=0.25)
    # plot.draw_med_conf_color(ax, xx, yy.reshape(NVARS, NREALS*NSKIES), color=colors[ii])

    # print(f"{yy.shape=}")

    if ii<4:
        file = np.load(path+f'/freq_meanvar_gw_{targets[ii]}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}.npz')
        xx_gw, yy_gw, vv_gw = file['xx'], file['yy']*YR, file['vv']*YR

    elif ii==4: # hard_time, get mean, hard_gamma_inner uses same
        yy_gw = np.repeat(yy_gw[10], NVARS)
        vv_gw = np.repeat(vv_gw[10], NVARS)

    ax.plot(xx, yy_gw, color='k', alpha=0.75, linestyle='--')
    # ax.errorbar()
    # for pp in [50,]:
    #     percs = [50-pp/2, 50+pp/2]
    #     ax.fill_between(xx, *np.percentile(yy_gw, percs, axis=-1), color='k', alpha=0.15, linestyle='--')
    
    if ii%2 == 1:
        _twiny_hz(ax, label=False)

    # print min and max freq
    print(f"{np.min(yy)=}, {np.max(yy)=}, {np.min(yy_gw)=}, {np.max(yy_gw)=}")
# fig.tight_layout()
    # ax.tick_params(axis='x', labelrotation=45)

fig.text(0.04, 0.5, ylabel, ha='right', va='center', rotation='vertical')
fig.text(1.09, 0.5, ylabel_nHz, ha='right', va='center', rotation='vertical')
plt.subplots_adjust(wspace=0.05, hspace=0.35)
# saveloc = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots'
# savename = saveloc+'/fmeanvar_gw_single.png'
# fig.savefig(savename, dpi=100, bbox_inches='tight')

In [ ]:

targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
    'mmb_mamp_log10', 'mmb_scatter_dex',
    'hard_time', 'hard_gamma_inner']
colors = [
    '#336948', '#336948', 
    '#9e5c41', '#9e5c41', 
    '#2d839f', '#2d839f',
]
path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata'  
   

ylabel = 'Frequency [yr$^{-1}$]'
ylabel_nHz = 'log( Frequency / [$\mathrm{yr}^{-1}$])'
fig, axs = plot.figax_single(nrows=3, ncols=2, 
                             sharey=True, sharex=False, xscale='linear',yscale='linear',
                             height=7)


for ii, ax in enumerate(axs.flatten()):
    ax.set_xlabel(plot.PARAM_KEYS[targets[ii]])
    # if ii == 0 or ii == 3:
    #     ax.set_ylabel(ylabel)
    file = np.load(path+f'/freq_meanvar_{targets[ii]}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}.npz')
    xx, yy, vv, yy_log, vv_log = file['xx'], file['yy'], file['vv'], file['yy_log'], file['vv_log']
    yy_min = (yy_log - vv_log) + np.log10(YR)
    yy_max = (yy_log + vv_log) + np.log10(YR)
    yy = yy_log + np.log10(YR)
    # print(f"{yy.shape=}")

    ax.plot(xx, yy, color=colors[ii], alpha=0.9)
    ax.errorbar(xx, yy, yerr=(yy-yy_min, yy_max-yy), color=colors[ii], alpha=0.9, capsize=3)
    # ax.errorbar(xx, yy, yerr=(yy_min, yy_max), color=colors[ii], alpha=0.7, capsize=3)
    # ax.scatter(xx, yy_min, color=colors[ii], alpha=0.7, marker=10)
    # ax.scatter(xx, yy_max, color=colors[ii], alpha=0.7, marker=11)

    # for pp in [50,95]:
    #     percs = [50-pp/2, 50+pp/2]
    #     ax.fill_between(xx, *np.percentile(yy, percs, axis=-1), color=colors[ii], alpha=0.25)
    # plot.draw_med_conf_color(ax, xx, yy.reshape(NVARS, NREALS*NSKIES), color=colors[ii])

    # print(f"{yy.shape=}")

    # if ii<4:
    #     file = np.load(path+f'/freq_meanvar_gw_{targets[ii]}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}.npz')
    #     xx_gw, yy_gw, vv_gw = file['xx'], file['yy']*YR, file['vv']*YR

    # elif ii==4: # hard_time, get mean, hard_gamma_inner uses same
    #     yy_gw = np.repeat(yy_gw[10], NVARS)
    #     vv_gw = np.repeat(vv_gw[10], NVARS)

    ax.plot(xx, yy_gw, color='k', alpha=0.75, linestyle='--')
    # ax.errorbar()
    # for pp in [50,]:
    #     percs = [50-pp/2, 50+pp/2]
    #     ax.fill_between(xx, *np.percentile(yy_gw, percs, axis=-1), color='k', alpha=0.15, linestyle='--')
    
    if ii%2 == 1:
        _twiny_hz(ax, label=False)

    # print min and max freq
    print(f"{np.min(yy)=}, {np.max(yy)=}, {np.min(yy_gw)=}, {np.max(yy_gw)=}")
# fig.tight_layout()
    # ax.tick_params(axis='x', labelrotation=45)

fig.text(0.04, 0.5, ylabel, ha='right', va='center', rotation='vertical')
fig.text(1.09, 0.5, ylabel_nHz, ha='right', va='center', rotation='vertical')
plt.subplots_adjust(wspace=0.05, hspace=0.35)
# saveloc = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots'
# savename = saveloc+'/fmeanvar_gw_single.png'
# fig.savefig(savename, dpi=100, bbox_inches='tight')

In [ ]:

targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
    'mmb_mamp_log10', 'mmb_scatter_dex',
    'hard_time', 'hard_gamma_inner']
colors = [
    '#336948', '#336948', 
    '#9e5c41', '#9e5c41', 
    '#2d839f', '#2d839f',
]
path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata'  
   

ylabel = 'Frequency [yr$^{-1}$]'
ylabel_nHz = 'Frequency [nHz]'
fig, axs = plot.figax_single(nrows=3, ncols=2, 
                             sharey=True, sharex=False, xscale='linear',  yscale='linear',
                             height=7)


for ii, ax in enumerate(axs.flatten()):
    ax.set_xlabel(plot.PARAM_KEYS[targets[ii]])
    # if ii == 0 or ii == 3:
    #     ax.set_ylabel(ylabel)
    file = np.load(path+f'/freq_meanvar_{targets[ii]}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}.npz')
    xx, yy, vv,  = file['xx'], file['yy']*YR, file['vv']*YR, 
    # yy_min = 10**(np.log10(yy) - vv_log)
    # yy_max = 10**(np.log10(yy) + vv_log)
    # print(f"{yy.shape=}")

    ax.plot(xx, yy, color=colors[ii], alpha=0.9)
    # ax.errorbar(xx, yy, yerr=(yy_min, yy_max), color=colors[ii], alpha=0.9)
    ax.errorbar(xx, yy, yerr=vv, color=colors[ii], alpha=0.9, capsize=3)

    # for pp in [50,95]:
    #     percs = [50-pp/2, 50+pp/2]
    #     ax.fill_between(xx, *np.percentile(yy, percs, axis=-1), color=colors[ii], alpha=0.25)
    # plot.draw_med_conf_color(ax, xx, yy.reshape(NVARS, NREALS*NSKIES), color=colors[ii])

    # print(f"{yy.shape=}")

    if ii<4:
        file = np.load(path+f'/freq_meanvar_gw_{targets[ii]}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}.npz')
        xx_gw, yy_gw, vv_gw = file['xx'], file['yy']*YR, file['vv']*YR

    elif ii==4: # hard_time, get mean, hard_gamma_inner uses same
        yy_gw = np.repeat(yy_gw[10], NVARS)
        vv_gw = np.repeat(vv_gw[10], NVARS)

    ax.plot(xx, yy_gw, color='k', alpha=0.75, linestyle='--')
    # ax.errorbar()
    # for pp in [50,]:
    #     percs = [50-pp/2, 50+pp/2]
    #     ax.fill_between(xx, *np.percentile(yy_gw, percs, axis=-1), color='k', alpha=0.15, linestyle='--')
    
    if ii%2 == 1:
        _twiny_hz(ax, label=False)

    # print min and max freq
    print(f"{np.min(yy)=}, {np.max(yy)=}, {np.min(yy_gw)=}, {np.max(yy_gw)=}")
# fig.tight_layout()
    # ax.tick_params(axis='x', labelrotation=45)
    # ax.set_ylim(10**-2, 4*10**-1)

fig.text(0.04, 0.5, ylabel, ha='right', va='center', rotation='vertical')
fig.text(1.09, 0.5, ylabel_nHz, ha='right', va='center', rotation='vertical')
plt.subplots_adjust(wspace=0.05, hspace=0.35)
# saveloc = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots'
# savename = saveloc+'/fmeanvar_linear_gw_single.png'
# fig.savefig(savename, dpi=100, bbox_inches='tight')

In [ ]:
def weighted_incremental_variance(data, weights):
    w_sum = w_sum2 = mean = S = 0

    for ii, xx, in enumerate(data):
        ww = weights[ii]
        w_sum = w_sum + ww
        w_sum2 = w_sum2 + ww**2
        mean_old = mean
        mean = mean_old + (ww / w_sum) * (xx - mean_old)
        S = S + ww * (xx - mean_old) * (xx - mean)

    population_variance = S / w_sum
    # Bessel's correction for weighted samples
    # Frequency weights
    sample_frequency_variance = S / (w_sum - 1)
    # Reliability weights
    sample_reliability_variance = S / (w_sum - w_sum2 / w_sum)

In [ ]:
target = 'gsmf_phi0'
path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata'  

print(target)
xx = [] # param
yy = [] # frequency means
vv = [] # variance
vv_log = [] # variance of log frequency
yy_log = []

# white noise only
data, params, dsdat = get_data(target)
freqs = data[0]['fobs_cents']*YR
freqs = np.repeat(freqs, NREALS*NSKIES*NLOUDEST).reshape(NFREQS, NREALS, NSKIES, NLOUDEST)

for pp, par in enumerate(params):
    xx.append(params[pp][target])
    dpssi = dsdat[pp]['gamma_ssi']
    mean, var2 = weighted_mean_variance(freqs, weights=dpssi)
    logmean, logvar2 = weighted_mean_variance(np.log10(freqs), weights=dpssi)
    var = np.sqrt(var2)
    logvar = np.sqrt(logvar2)
    yy.append(mean)
    vv.append(var)
    vv_log.append(logvar)
    yy_log.append(logmean)

print(f"{vv=}")
print(f"{vv_log=}")

In [ ]:
fig, ax = plot.figax(xlabel=plot.PARAM_KEYS['gsmf_phi0'], ylabel=plot.LABEL_GW_FREQUENCY_YR, xscale='linear')


yy = np.array(yy)                    # yy = mean[frequencies]
yy_log = np.array(yy_log)            # yy_log = mean[log10 frequencies]
vv = np.array(vv_log)                # vv = sqrt(variance[frequencies])
vv_log - np.array(vv_log)            # vv_log = sqrt(variance[log10 frequencies])


ymin = 10**(np.log10(yy-vv)) 
ymax = 10**(np.log10(yy+vv))
ax.errorbar(xx, yy, yerr=(ymin, ymax), label='version 1A', alpha=0.5, capsize=3, marker='*')
print(f"V1A: {ymin=}, \nV1A: {ymax=}")

ymin = 10**(np.log10(yy) / np.log10(vv))
ymax = 10**(np.log10(yy) * np.log10(vv))
ax.errorbar(xx, yy, yerr=(ymin, ymax), label='version 1B', alpha=0.5, capsize=3, marker='d')
print(f"V1B: {ymin=}, \nV1B: {ymax=}")

ymin = 10**(np.log10(yy) - vv_log)
ymax = 10**(np.log10(yy) + vv_log)
ax.errorbar(xx, yy, yerr=(ymin, ymax), label='version 2A', alpha=0.5, capsize=3, marker='x')
print(f"V2A: {ymin=}, \nV2A: {ymax=}")

ymin = 10**(yy_log - vv_log)
ymax = 10**(yy_log + vv_log)
ax.errorbar(xx, 10**yy_log, yerr=(ymin, ymax), label='version 2B', alpha=0.5, capsize=3, marker='+')
print(f"V2B: {ymin=}, \nV2B: {ymax=}")

ymin = 10**(np.log10(yy) - np.log10(vv))
ymax = 10**(np.log10(yy) + np.log10(vv))
ax.errorbar(xx, yy, yerr=(ymin, ymax), label='version 3', alpha=0.5, capsize=3, marker='o')
print(f"V3: {ymin=}, \nV3: {ymax=}")



ax.legend()
# ax.set_ylim(10**-30, 10**17)

In [ ]:
# print(f"{vv*YR=}")